<a href="https://colab.research.google.com/github/aya-rhouma/InvestmentCo-PilotAINightChallenge/blob/main/InvestmentCo_Pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
np.random.seed(42)
n_stocks = 50
sectors = ['Technologie', 'Santé', 'Finance', 'Énergie', 'Consommation', 'Industrie', 'Télécom']
stocks = []
for i in range(n_stocks):
    sector = np.random.choice(sectors)
    price = np.random.uniform(20, 500)
    change = np.random.uniform(-10, 10)
    pe = np.random.uniform(8, 30)
    stocks.append({
        'Symbol': f'STK{i:03d}',
        'Name': f'Company {i}',
        'Sector': sector,
        'Price': round(price, 2),
        'Change_%': round(change, 2),
        'PER': round(pe, 2),
        'Market_Cap_B': round(price * np.random.uniform(10, 1000), 2)
    })
stocks_df = pd.DataFrame(stocks)

In [3]:
news = []
for i in range(200):
    symbols = np.random.choice(stocks_df['Symbol'], size=np.random.randint(1,4), replace=False)
    titles = [
        f"{np.random.choice(['Baisse', 'Hausse', 'Stabilité'])} des actions de {', '.join(symbols)}",
        f"Analyse du secteur {np.random.choice(sectors)}",
        f"Prévisions de marché pour {np.random.choice(stocks_df['Name'])}",
        f"Nouveau rapport trimestriel : {np.random.choice(['bénéfices en hausse', 'pertes', 'dividende'])}"
    ]
    news.append({
        'Date': (datetime.now() - timedelta(days=np.random.randint(0,30))).strftime('%Y-%m-%d'),
        'Title': np.random.choice(titles),
        'Summary': f"Ceci est un résumé de l'actualité concernant {', '.join(symbols)}. {np.random.choice(['Les analystes sont optimistes.', 'Les perspectives sont mitigées.', 'Le marché réagit positivement.'])}",
        'Symbols': ', '.join(symbols)
    })
news_df = pd.DataFrame(news)

In [4]:
!pip install sentence-transformers faiss-cpu gradio pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 65.4 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
stock_texts = stocks_df.apply(lambda row: f"{row['Name']} ({row['Symbol']}) est une société du secteur {row['Sector']}. Son prix est de {row['Price']} USD, avec une variation de {row['Change_%']}% et un PER de {row['PER']}.", axis=1).tolist()
news_texts = news_df.apply(lambda row: f"Actualité du {row['Date']}: {row['Title']} - {row['Summary']} (symboles: {row['Symbols']})", axis=1).tolist()
all_texts = stock_texts + news_texts

In [13]:
embeddings = model.encode(all_texts, show_progress_bar=True)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(embeddings)
index.add(embeddings.astype(np.float32))
print(f"Index FAISS créé avec {len(all_texts)} documents.")

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Index FAISS créé avec 250 documents.


In [14]:
def search(query, top_k=5):
    query_emb = model.encode([query])
    faiss.normalize_L2(query_emb)
    scores, indices = index.search(query_emb.astype(np.float32), top_k)
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(stock_texts):
            source = 'stock'
            text = stock_texts[idx]
            metadata = stocks_df.iloc[idx].to_dict()
        else:
            source = 'news'
            news_idx = idx - len(stock_texts)
            text = news_texts[news_idx]
            metadata = news_df.iloc[news_idx].to_dict()
        results.append((text, scores[0][i], source, metadata))
    return results


In [15]:
def answer(query):
    results = search(query, top_k=3)
    context = "\n\n".join([f"- {r[0]}" for r in results])
    response = f"**Informations pertinentes :**\n\n{context}\n\n**Conseil :** En fonction de ces données, il est recommandé de diversifier votre portefeuille et de consulter un conseiller financier."
    return response

In [20]:
import gradio as gr
import pandas as pd
import plotly.express as px
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from datetime import datetime, timedelta


def interactive_answer(query, top_k):
    results = search(query, top_k)
    df_results = pd.DataFrame([
        {
            'Source': 'Action' if r[2]=='stock' else 'Actualité',
            'Extrait': r[0][:120] + '...',
            'Score': round(r[1], 3)
        } for r in results
    ])
    context = "\n\n".join([f"- {r[0]}" for r in results])
    response = f"**Informations pertinentes :**\n\n{context}\n\n**Conseil :** En fonction de ces données, il est recommandé de diversifier votre portefeuille et de consulter un conseiller financier."
    sector_counts = stocks_df['Sector'].value_counts().reset_index()
    sector_counts.columns = ['Secteur', 'Nombre']
    fig = px.bar(sector_counts, x='Secteur', y='Nombre', title="Répartition des actions par secteur",
                 color='Secteur', color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_layout(showlegend=False)
    return response, df_results, fig

with gr.Blocks(title="Investment Co-Pilot", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📈 Investment Co-Pilot")
    gr.Markdown("Assistant IA pour les marchés financiers (données simulées). Posez une question et obtenez des informations pertinentes.")

    with gr.Row():
        with gr.Column(scale=4):
            query_input = gr.Textbox(
                label="Votre question",
                placeholder="Ex: Quelles actions technologiques ont un faible PER ?",
                lines=2
            )
        with gr.Column(scale=1):
            top_k_slider = gr.Slider(
                minimum=1, maximum=10, value=5, step=1,
                label="Nombre de résultats"
            )

    with gr.Row():
        submit_btn = gr.Button("🔍 Rechercher", variant="primary", scale=0)

    with gr.Row():
        with gr.Column():
            response_output = gr.Markdown(label="Réponse")
        with gr.Column():
            table_output = gr.Dataframe(label="Résultats détaillés")

    with gr.Row():
        plot_output = gr.Plot(label="Statistiques du marché")

    gr.Examples(
        examples=[
            ["Actions du secteur Technologie avec PER < 15"],
            ["Actualités récentes sur la santé"],
            ["Que penser de l'action STK005 ?"],
            ["Quelles sont les meilleures actions du moment ?"],
        ],
        inputs=query_input
    )

    submit_btn.click(
        fn=interactive_answer,
        inputs=[query_input, top_k_slider],
        outputs=[response_output, table_output, plot_output]
    )

demo.launch(share=True)

/tmp/ipython-input-298/1035498884.py:28: DeprecationWarning:

The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ace2f178a2189172cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
